Copyright **`(c)`** 2023 Giovanni Squillero `<giovanni.squillero@polito.it>`  
[`https://github.com/squillero/computational-intelligence`](https://github.com/squillero/computational-intelligence)  
Free for personal or classroom use; see [`LICENSE.md`](https://github.com/squillero/computational-intelligence/blob/master/LICENSE.md) for details.  

# LAB9

Write a local-search algorithm (eg. an EA) able to solve the *Problem* instances 1, 2, 5, and 10 on a 1000-loci genomes, using a minimum number of fitness calls. That's all.

### Deadlines:

* Submission: Sunday, December 3 ([CET](https://www.timeanddate.com/time/zones/cet))
* Reviews: Sunday, December 10 ([CET](https://www.timeanddate.com/time/zones/cet))

Notes:

* Reviews will be assigned  on Monday, December 4
* You need to commit in order to be selected as a reviewer (ie. better to commit an empty work than not to commit)

In [250]:
from random import choices, random, randint
import lab9_lib

In [251]:
pop = 1
fitness = lab9_lib.make_problem(pop)
for n in range(pop):
    ind = choices([0, 1], k=50)
    print(f"{''.join(str(g) for g in ind)}: {fitness(ind):.2%}")

print(fitness.calls)

10101100110110111110000001001111110001100100100101: 52.00%
1


### Local Search EA:
The algorithm runs for a specified number of generation, evolving the population to improve the best fitness.
In each generation:
- Parents are selected from the current population using the `select` method
- Cross-over is applied to the parents to generate the offspring
- Mutation is applied to the offspring to generate the mutated offspring
    - mutation strategy: each gene is mutated to 0 or 1 based initialty on a 50/50 probability, that is tuned based on the current best fitness (decrese the mutation rate if the fitness increase with 1, otherwise increase the mutation rate)
    - this strategy is applied to give the genome a direction, instead of randomly mutating the genome
- The worst individuals (based on fitness) are replaced by the mutated offspring

To reduce the number of fitness calls, the algorithm keeps track of the stagnation of the best fitness. If the best fitness does not improve for a specified number of generations, the algorithm stops. Also, if the best fitness arrives at a certain threshold, the algorithm stops.

In [252]:
from random import choices, random, randint

class LocalSearchEA:
    def __init__(self, fitness_function, population_size=10, num_generations=100, mutation_rate=0.5, max_stagnation=20):
        self.fitness_function = fitness_function
        self.population_size = population_size
        self.num_generations = num_generations
        self.mutation_rate = mutation_rate
        self.max_stagnation = max_stagnation
    
    def initialize_population(self):
        k = 1000
        return [choices([0, 1], k=k) for _ in range(self.population_size)]

    def mutate(self, individual):
        # Too achive minimum call to fitness function we can use previous results (from previous test, with more random strategy) to tweak the mutation in a way to improve the fitness function quickly, 
        # from previous runs we discovered that the best way to increase the fitness is changing the gene to 1.
        # To retain the random mutation I decided to start with a mutation rate of 0.5 and then change it based on the fitness function (increase or decrease by 10%)
        for gene in range(len(individual)):
            if random() < self.mutation_rate:
                individual[gene] = 1
            else:
                individual[gene] = 0
        return individual

    def crossover(self, parent1, parent2):
        k = randint(1, len(parent1) - 1)
        child1 = parent1[:k] + parent2[k:]
        child2 = parent2[:k] + parent1[k:]
        return child1, child2

    def select(self, population):
        return choices(population, k=2, weights=[self.fitness_function(individual) for individual in population])

    def run(self):
        population = self.initialize_population()
        best_individual = max(population, key=self.fitness_function)
        best_fitness = self.fitness_function(best_individual)
        stagnation_count = 0

        for generation in range(1, self.num_generations + 1):
            paren1, parent2 = self.select(population)
            child1, child2 = self.crossover(paren1, parent2)
            child1 = self.mutate(child1)
            child2 = self.mutate(child2)

            # Replace worst individual with best child
            worst_individual = min(population, key=self.fitness_function)
            population.remove(worst_individual)
            population.extend([child1, child2])

            current_best_individual = max(population, key=self.fitness_function)
            current_best_fitness = self.fitness_function(current_best_individual)

            if current_best_fitness > best_fitness:
                best_individual = current_best_individual
                best_fitness = current_best_fitness
                stagnation_count = 0
                self.mutation_rate = self.mutation_rate * 0.9
            else:
                stagnation_count += 1
                self.mutation_rate = self.mutation_rate * 1.1

            if stagnation_count >= self.max_stagnation:
                print(f"Stopping early after {generation} generations due to stagnation.")
                break

            if best_fitness >= 0.99:
                print(f"Stopping early after {generation} generations due to fitness convergence.")
                break

        print(f"fitness: {best_fitness},\nfitness calls: {self.fitness_function.calls}")


### Local Search EA with memoization:
This version of the algorithm use a memoization technique to reduce the number of fitness calls. The memoization is implemented using a dictionary that maps the individuals to their fitness. The dictionary is updated at each fitness call, and the fitness is retrieved from the dictionary if the individual is already present, otherwise the fitness is computed and the individual is added to the dictionary.
The rest of the algorithm is the same as the previous one.

In [253]:
from random import choices, random, randint

class LocalSearchEA_with_memoization:
    def __init__(self, fitness_function, population_size=10, num_generations=100, mutation_rate=0.5, max_stagnation=20):
        self.fitness_function = fitness_function
        self.population_size = population_size
        self.num_generations = num_generations
        self.mutation_rate = mutation_rate
        self.max_stagnation = max_stagnation
        self.memoized_fitness = {}
    
    def initialize_population(self):
        k = 1000
        return [choices([0, 1], k=k) for _ in range(self.population_size)]
    
    def get_fitness(self, individual):
        if tuple(individual) not in self.memoized_fitness:
            fitness_value = self.fitness_function(individual)
            self.memoized_fitness[tuple(individual)] = fitness_value
        return self.memoized_fitness[tuple(individual)]

    def mutate(self, individual):
        # Too achive minimum call to fitness function we can use previous results to tweak the mutation in a way to improve the fitness function quickly, 
        # from prvious runs we discovered that the best way to do that is changing the gene to 1.
        # To retain the random mutation I decided to start with a mutation rate of 0.5 and then change it based on the fitness function (increase or decrease by 10%)
        for gene in range(len(individual)):
            if random() < self.mutation_rate:
                individual[gene] = 1
            else:
                individual[gene] = 0
        return individual

    def crossover(self, parent1, parent2):
        k = randint(1, len(parent1) - 1)
        child1 = parent1[:k] + parent2[k:]
        child2 = parent2[:k] + parent1[k:]
        return child1, child2

    def select(self, population):
        return choices(population, k=2, weights=[self.get_fitness(individual) for individual in population])

    def run(self):
        population = self.initialize_population()
        best_individual = max(population, key=self.get_fitness)
        best_fitness = self.get_fitness(best_individual)
        stagnation_count = 0

        for generation in range(1, self.num_generations + 1):
            paren1, parent2 = self.select(population)
            child1, child2 = self.crossover(paren1, parent2)
            child1 = self.mutate(child1)
            child2 = self.mutate(child2)

            # Replace worst individual with best child
            worst_individual = min(population, key=self.get_fitness)
            population.remove(worst_individual)
            population.extend([child1, child2])

            current_best_individual = max(population, key=self.get_fitness)
            current_best_fitness = self.get_fitness(current_best_individual)

            if current_best_fitness > best_fitness:
                best_individual = current_best_individual
                best_fitness = current_best_fitness
                stagnation_count = 0
                self.mutation_rate = self.mutation_rate * 0.9
            else:
                stagnation_count += 1
                self.mutation_rate = self.mutation_rate * 1.1

            if stagnation_count >= self.max_stagnation:
                print(f"Stopping early after {generation} generations due to stagnation.")
                break

            if best_fitness >= 0.99:
                print(f"Stopping early after {generation} generations due to fitness convergence.")
                break

        print(f"fitness: {best_fitness},\nfitness calls: {self.fitness_function.calls}")


### Parameters:
Parameters are set by default, could be changed to get better results. 
- MUTATION_RATE: probability of mutation per gene in an individual
- NUM_GENERATIONS: number of generations to run the algorithm for before stopping, coul be increase to better results, but it will take more time
- MAX_STAGNATION: number of generations to run the algorithm without improvement before stopping, could be increased to better results, but it will take more time
- POPULATION_SIZE: I've decided to use the instances as a population size, but setting to a fixed value like 100 could be better in terms of generations but it will take more fitness calls

In [254]:
MUTATION_RATE = 0.5
NUM_GENERATIONS = 1000
MAX_STAGNATION = 100

In [255]:
instances = 1
print(f"Local Search evolutionary algorithm with {instances} instance(s)")
fitness = lab9_lib.make_problem(instances)
ea = LocalSearchEA(fitness, 
                   population_size=instances, 
                   num_generations=NUM_GENERATIONS, 
                   mutation_rate=MUTATION_RATE, 
                   max_stagnation=MAX_STAGNATION)
ea.run()
print("-----------------------------------------------")
print(f"Local Search evolutionary algorithm with {instances} instance(s) and memoization")
fitness_with_memoization = lab9_lib.make_problem(instances)
ea = LocalSearchEA_with_memoization(fitness_with_memoization,
                                    population_size=instances, 
                                    num_generations=NUM_GENERATIONS, 
                                    mutation_rate=MUTATION_RATE, 
                                    max_stagnation=MAX_STAGNATION)
ea.run()

Local Search evolutionary algorithm with 1 instance(s)
Stopping early after 30 generations due to fitness convergence.
fitness: 1.0,
fitness calls: 1457
-----------------------------------------------
Local Search evolutionary algorithm with 1 instance(s) and memoization
Stopping early after 30 generations due to fitness convergence.
fitness: 1.0,
fitness calls: 60


In [256]:
instances = 2
print(f"Local Search evolutionary algorithm with {instances} instance(s)")
fitness = lab9_lib.make_problem(instances)
ea = LocalSearchEA(fitness, 
                   population_size=instances, 
                   num_generations=NUM_GENERATIONS, 
                   mutation_rate=MUTATION_RATE, 
                   max_stagnation=MAX_STAGNATION)
ea.run()
print("-----------------------------------------------")
print(f"Local Search evolutionary algorithm with {instances} instance(s) and memoization")
fitness_with_memoization = lab9_lib.make_problem(instances)
ea = LocalSearchEA_with_memoization(fitness_with_memoization,
                                    population_size=instances, 
                                    num_generations=NUM_GENERATIONS, 
                                    mutation_rate=MUTATION_RATE, 
                                    max_stagnation=MAX_STAGNATION)
ea.run()

Local Search evolutionary algorithm with 2 instance(s)
Stopping early after 15 generations due to fitness convergence.
fitness: 1.0,
fitness calls: 438
-----------------------------------------------
Local Search evolutionary algorithm with 2 instance(s) and memoization
Stopping early after 17 generations due to fitness convergence.
fitness: 1.0,
fitness calls: 35


In [257]:
instances = 5
print(f"Local Search evolutionary algorithm with {instances} instance(s)")
fitness = lab9_lib.make_problem(instances)
ea = LocalSearchEA(fitness, 
                   population_size=instances, 
                   num_generations=NUM_GENERATIONS, 
                   mutation_rate=MUTATION_RATE, 
                   max_stagnation=MAX_STAGNATION)
ea.run()
print("-----------------------------------------------")
print(f"Local Search evolutionary algorithm with {instances} instance(s) and memoization")
fitness_with_memoization = lab9_lib.make_problem(instances)
ea = LocalSearchEA_with_memoization(fitness_with_memoization,
                                    population_size=instances, 
                                    num_generations=NUM_GENERATIONS, 
                                    mutation_rate=MUTATION_RATE, 
                                    max_stagnation=MAX_STAGNATION)
ea.run()

Local Search evolutionary algorithm with 5 instance(s)
Stopping early after 13 generations due to fitness convergence.
fitness: 1.0,
fitness calls: 461
-----------------------------------------------
Local Search evolutionary algorithm with 5 instance(s) and memoization
Stopping early after 13 generations due to fitness convergence.
fitness: 1.0,
fitness calls: 30


In [258]:
instances = 10
print(f"Local Search evolutionary algorithm with {instances} instance(s)")
fitness = lab9_lib.make_problem(instances)
ea = LocalSearchEA(fitness, 
                   population_size=instances, 
                   num_generations=NUM_GENERATIONS, 
                   mutation_rate=MUTATION_RATE, 
                   max_stagnation=MAX_STAGNATION)
ea.run()
print("-----------------------------------------------")
print(f"Local Search evolutionary algorithm with {instances} instance(s) and memoization")
fitness_with_memoization = lab9_lib.make_problem(instances)
ea = LocalSearchEA_with_memoization(fitness_with_memoization,
                                    population_size=instances, 
                                    num_generations=NUM_GENERATIONS, 
                                    mutation_rate=MUTATION_RATE, 
                                    max_stagnation=MAX_STAGNATION)
ea.run()

Local Search evolutionary algorithm with 10 instance(s)
Stopping early after 15 generations due to fitness convergence.
fitness: 1.0,
fitness calls: 806
-----------------------------------------------
Local Search evolutionary algorithm with 10 instance(s) and memoization
Stopping early after 13 generations due to fitness convergence.
fitness: 1.0,
fitness calls: 35
